# **Trabajo de Fin de Máster**

# **1. Imports**

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

from datasets import load_dataset

import torch 

# Usual imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy import displacy

# **2. Definición de funciones**

## **2.1. Carga del texto de entrada**

In [2]:
def load_corpus(data_files, ext_type='csv'):
    '''
    '''
    return load_dataset(ext_type, data_files=data_files)

In [3]:
def filter_corpus(corpus, element, filter_with):
    '''
    '''
    return corpus.filter(lambda filtered_corpus: filtered_corpus[element].startswith(filter_with))

In [4]:
def get_script_episode(corpus, season, episode):
    '''
    '''
    data_season = filter_corpus(corpus,'Season', season)
    return filter_corpus(data_season,'Episode', episode)

In [5]:
def get_full_text(input_text):
    return ''.join(input_text)

## **2.2. Procesamiento del texto de entrada**

In [6]:
def get_summary(input_text, model_name='sshleifer/distilbart-cnn-12-6'):
    '''
    Models: "sshleifer/distilbart-xsum-12-3", "google/pegasus-xsum"
    '''
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    # encode input context
    input_ids = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt").input_ids
    # generate summary
    outputs = model.generate(input_ids=input_ids)
    # decode summary 
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
def show_entities(input_text):
    displacy.render(nlp(input_text), style="ent")

In [8]:
def spacy_tokenizer(sentence):
    parser = English()
    mytokens = parser(sentence)
    mytokens = [word.lower_ for word in mytokens]
    #mytokens = [word.lemma_.lower().strip() if word.lemma_ != "PRON" else word.lower_ for word in mytokens]
    mytokens = [word for word in mytokens if word not in stopwords and word not in punctuations]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

# **3. Procesamiento de un script de ejemplo de _Game of Thrones_**

*Source: https://www.kaggle.com/thebrownviking20/topic-modelling-with-spacy-and-scikit-learn/notebook?select=winemag-data_first150k.csv*

In [9]:
# Loading data
corpus = load_corpus("qubartCode/corpus/Game_Of_Thrones_Script.csv")['train']
s1_ep2 = get_full_text(get_script_episode(corpus, 'Season 1', 'Episode 2')['Sentence'])
s1_ep2_names = ''
for row in get_script_episode(corpus, 'Season 1', 'Episode 2'):
    line = f"{row['Name'].capitalize()}: {row['Sentence']}"
    s1_ep2_names = s1_ep2_names + " " + line
corpus_reviews = load_corpus("qubartCode/corpus/GameOfThrones_Reviews.csv")['train']

Using custom data configuration default-6267e762714f0fd0
Reusing dataset csv (/Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-05842c71d6ab9f3e.arrow
Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-80a1cce4dbd0b731.arrow
Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-05842c71d6ab9f3e.arrow
Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304

In [60]:
corpus_reviews['Episode recap'][30]

'Another wedding, another horrific death scene. The shocking ending of “The Lion and the Rose” sets up the Lannister family breakdown that plays out over the rest of season 4.In the woods surrounding the Dreadfort, Ramsay forces Theon, who he now exclusively refers to as Reek, to accompany him and his lover, Myranda (Charlotte Hope), on a hunt with his dogs. However, much to Theon’s horror, the prey is not an animal, but rather a servant girl that Ramsay and Myranda have grown tired of playing with.Later, Roose Bolton arrives at the Dreadfort with his new wife Walda (Elizabeth Webster), a granddaughter of Walder Frey, and Locke. At first, Roose is infuriated that Ramsay has both physically and psychologically tortured Theon, who Roose sees as a valuable hostage. But when Ramsay proves that he has completely broken Theon by revealing to him that Robb was murdered at the Red Wedding while allowing Theon to shave him with a sharp razor, Roose seems to come around to Ramsay’s methods. Rams

In [11]:
corpus

Dataset({
    features: ['Release Date', 'Season', 'Episode', 'Episode Title', 'Name', 'Sentence'],
    num_rows: 23911
})

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
# Creating a spaCy object
nlp = spacy.load('en_core_web_lg')

In [14]:
doc = nlp(corpus_reviews['Episode recap'][1])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [15]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [16]:
review = str(" ".join([i.lemma_ for i in doc]))

In [17]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [18]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

although => SCONJ
Jon => PROPN
Snow => PROPN
( => PUNCT
Kit => PROPN
Harington => PROPN
) => PUNCT
, => PUNCT
who => PRON
we => PRON
know => VERB
as => ADP
the => DET
illegitimate => ADJ
son => NOUN
of => ADP
Ned => PROPN
and => CCONJ
a => DET
random => ADJ
, => PUNCT
unknown => ADJ
woman => NOUN
, => PUNCT
be => VERB
introduce => VERB
in => ADP
the => DET
pilot => NOUN
, => PUNCT
his => PRON
storyline => NOUN
do => AUX
n’t => AUX
really => ADV
pick => VERB
up => ADP
until => ADP
the => DET
second => ADJ
episode => NOUN
. => PUNCT
after => ADP
gift => NOUN
his => PRON
half => ADJ
- => PUNCT
sister => NOUN
Arya => PROPN
( => PUNCT
Maisie => PROPN
Williams => PROPN
) => PUNCT
with => ADP
a => DET
small => ADJ
sword => NOUN
that => DET
she => PRON
dub => VERB
" => PUNCT
Needle => PROPN
, => PUNCT
" => PUNCT
Jon => PROPN
leave => VERB
for => ADP
the => DET
Wall => PROPN
, => PUNCT
a => DET
700 => NUM
- => PUNCT
foot => NOUN
tall => ADJ
, => PUNCT
300 => NUM
- => PUNCT
foot => NOUN
thick =>

In [19]:
tqdm.pandas()
tokens = spacy_tokenizer(corpus_reviews['Episode recap'][1])

In [20]:
tokens

'jon snow kit harington know illegitimate son ned random unknown woman introduced pilot storyline pick second episode gifting half sister arya maisie williams small sword dubs “ needle ” jon leaves wall 700 foot tall 300 foot thick barrier ice spans 300 miles westeros ’ northern border uncle benjen joseph mawle tyrion lannister peter dinklage \n\n jon intends join night watch order men guard wall protect seven kingdoms invaders makes unsuccessful attempt question ned identity mother ahead ned departure king landing traveling south father arya sansa stark sophie turner find midst altercation prince joffrey baratheon jack gleeson leads deaths arya friend mycah rhodri hosking butcher boy sansa direwolf lady luckily arya able set direwolf nymeria loose wild harm come daenerys finds way connect new husband bran wakes coma month'

In [21]:
# Creating a vectorizer
vectorizer = CountVectorizer(stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform([tokens])

In [22]:
NUM_TOPICS = 10

In [23]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [24]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [25]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [26]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [27]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('know', 0.32996451215825806), ('kingdoms', 0.3234508732876145), ('sword', 0.3195542663274661), ('prince', 0.31495504807990427), ('invaders', 0.3146268228738855), ('connect', 0.30842463093997063), ('jon', 0.3073987410817163), ('baratheon', 0.3050032775729154), ('pick', 0.29936257340314276), ('sophie', 0.29707519451381664)]
Topic 1:
[('wall', 0.3967716801067743), ('arya', 0.38708669520936684), ('foot', 0.36842404380469723), ('illegitimate', 0.3514442709626562), ('jack', 0.3479312290786512), ('boy', 0.3472508360155583), ('random', 0.3461829524021131), ('ice', 0.3442926278189297), ('woman', 0.3405086842067669), ('snow', 0.3401154750964334)]
Topic 2:
[('wall', 0.3186916143557785), ('leads', 0.3107940366877808), ('jack', 0.30996237237460916), ('join', 0.3087370525782207), ('ahead', 0.30801473798107537), ('miles', 0.30611573319077995), ('month', 0.30406880031849365), ('joffrey', 0.3017828543080466), ('sophie', 0.3016511437313252), ('boy', 0.2998149067753576)]
Topic 3:
[(

In [28]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('arya', 1.5784119884114536), ('ned', 1.4771217861737826), ('jon', 1.3610094121659495), ('sansa', 0.7380849974508651), ('foot', 0.7214838734464727), ('direwolf', 0.6796656419079941), ('wall', 0.638398406562371), ('butcher', 0.42024647527111114), ('luckily', 0.41273455573016987), ('storyline', 0.41091004950464005)]
Topic 1:
[('daenerys', 0.7552513750257647), ('random', 0.7308399799467883), ('protect', 0.707708795389756), ('lady', 0.6998632139446398), ('landing', 0.6789466245087998), ('sword', 0.6316656864931232), ('nymeria', 0.6165856499876812), ('deaths', 0.6125604848517888), ('sister', 0.577101493180137), ('attempt', 0.5743045615556722)]
Topic 2:
[('coma', 0.9522333664549272), ('night', 0.8173886269475857), ('king', 0.7142191680671534), ('ice', 0.7127940654824797), ('able', 0.6690417117653791), ('foot', 0.6673975071746274), ('midst', 0.6588910717001462), ('dubs', 0.6271865838227952), ('northern', 0.6264767911482906), ('harm', 0.6186860908400459)]
Topic 3:
[('mycah

In [29]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('arya', 0.3244428422615251), ('ned', 0.2433321316961438), ('jon', 0.2433321316961438), ('wall', 0.16222142113076254), ('foot', 0.16222142113076254), ('direwolf', 0.16222142113076254), ('sansa', 0.16222142113076254), ('kit', 0.08111071056538127), ('illegitimate', 0.08111071056538127), ('half', 0.08111071056538127)]


In [30]:
# Transforming an individual sentence
text = spacy_tokenizer("Arya Stark.")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.03333429 0.03333531 0.03333412 0.03333423 0.03333467 0.03333475
 0.03333539 0.69998814 0.0333346  0.03333449]


In [31]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
#dash

/Users/andreea/miniconda3/envs/tfm/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/andreea/miniconda3/envs/tfm/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/andreea/miniconda3/envs/tfm/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/andreea/miniconda3/envs/tfm/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/andreea/miniconda3/envs/tfm/lib/pytho

In [32]:
corpus_reviews['Episode recap'][1]

'Although Jon Snow (Kit Harington), who we know as the illegitimate son of Ned and a random, unknown woman, was introduced in the pilot, his storyline doesn’t really pick up until the second episode. After gifting his half-sister Arya (Maisie Williams) with a small sword that she dubs “Needle,” Jon leaves for the Wall, a 700-foot tall, 300-foot thick barrier of ice that spans all 300 miles of Westeros’ northern border, with his Uncle Benjen (Joseph Mawle) and Tyrion Lannister (Peter Dinklage).\n\nJon intends to join the Night’s Watch, an order of men who guard the Wall and protect the Seven Kingdoms from invaders. But first, he makes one last (unsuccessful) attempt to question Ned about the identity of his mother ahead of Ned’s departure for King’s Landing.While traveling south with their father, Arya and Sansa Stark (Sophie Turner) find themselves in the midst of an altercation with Prince Joffrey Baratheon (Jack Gleeson) that leads to the deaths of both Arya’s friend Mycah (Rhodri Ho

In [33]:
!pip install sentencepiece

In [34]:
#get_summary(corpus_reviews['Episode recap'][1])

In [35]:
#corpus_reviews['Episode recap'][2]

In [36]:
#corpus_reviews['Episode recap'][71]

In [37]:
#get_summary(corpus_reviews['Episode recap'][2])

In [38]:
#get_summary(corpus_reviews['Episode recap'][2], "sshleifer/distilbart-xsum-12-3")

In [39]:
#get_summary(corpus_reviews['Episode recap'][2], "google/pegasus-xsum")

### Usando script sin nombres 

In [40]:
#get_summary(s1_ep2)

### Usando script con nombres

In [41]:
#get_summary(s1_ep2_names)

In [42]:
#get_summary(corpus_reviews['Episode recap'][3], "hyunwoongko/ctrlsum-cnndm")

## **Pruebas con el sentence similarity**
https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

In [43]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    '''
    '''
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings, or the last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_sentence_embeddings(input_data, model_name='sentence-transformers/paraphrase-mpnet-base-v2'):
    '''
    '''
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    encoded_input = tokenizer(input_data, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        
    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    return sentence_embeddings.detach().numpy()

def get_similar_sentences(sentence, other_sentences, similarity_threshold=0.7):
    '''
    '''
    sims = cosine_similarity(sentence, other_sentences)
    return np.where(sims.reshape(-1) >= similarity_threshold)[0]

## Con corpus de scripts

In [44]:
# convert from PyTorch tensor to numpy array
sentences = get_script_episode(corpus, 'Season 1', 'Episode 2')['Sentence']
embedded_script = get_sentence_embeddings(sentences)

Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-05842c71d6ab9f3e.arrow
Loading cached processed dataset at /Users/andreea/.cache/huggingface/datasets/csv/default-6267e762714f0fd0/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-80a1cce4dbd0b731.arrow


In [45]:
from sklearn.metrics.pairwise import cosine_similarity
# calculate
idx_sentence = 26
similar_sentences = get_similar_sentences([embedded_script[idx_sentence]], embedded_script)

In [46]:
sentences[idx_sentence]

'Is Bran going to die?'

In [47]:
for idx in similar_sentences:
    print(sentences[idx])

Is Bran going to die?
You've said goodbye to Bran? He's not going to die. I know it.
What if he's wrong? Bran needs me.


In [48]:
similar_sentences = get_similar_sentences(get_sentence_embeddings("Is Bran not going to be alive anymore?"), embedded_script)

In [49]:
for idx in similar_sentences:
    print(sentences[idx])

Is Bran going to die?
You've said goodbye to Bran? He's not going to die. I know it.


In [50]:
similar_sentences = get_similar_sentences(get_sentence_embeddings("Bran"), embedded_script, similarity_threshold=0.3)

for idx in similar_sentences:
    print(sentences[idx])

Is Bran going to die?
I came to say goodbye to Bran.
You've said goodbye to Bran? He's not going to die. I know it.
What if he's wrong? Bran needs me.
What I am about to tell you must remain between us. I don't think Bran fell from that tower. I think he was thrown.
What about Bran?
I have prayed to the Seven for more than a month. Bran's life is in their hands now.


In [53]:
filtered_text = ""
for idx in similar_sentences:
    filtered_text = filtered_text + " " + sentences[idx]
print(filtered_text)

 Is Bran going to die? I came to say goodbye to Bran. You've said goodbye to Bran? He's not going to die. I know it. What if he's wrong? Bran needs me. What I am about to tell you must remain between us. I don't think Bran fell from that tower. I think he was thrown. What about Bran? I have prayed to the Seven for more than a month. Bran's life is in their hands now.


In [54]:
get_summary(filtered_text)

[" Is Bran going to die? I came to say goodbye to Bran? I don't think Bran fell from that tower. I think he was thrown. Bran needs me. What I am about to tell you must remain between us. I have prayed to the Seven for more than a month. Bran's life is in their hands now."]

In [56]:
get_summary(filtered_text, "google/pegasus-xsum")

["You've said goodbye to Bran."]

In [57]:
get_summary(filtered_text, "hyunwoongko/ctrlsum-cnndm")

[' Is Bran going to die?']

## Con corpus de Reviews

In [101]:
sentences = corpus_reviews['Episode recap'][30].split('.')
embedded_script_2 = get_sentence_embeddings(sentences)

In [102]:
similar_sentences = get_similar_sentences(get_sentence_embeddings("Bran"), embedded_script_2, similarity_threshold=0.3)

In [103]:
for idx in similar_sentences:
    print(sentences[idx])



Meanwhile, Bran is struggling with the increasingly blurred lines between his own mind and that of his direwolf, Summer, as he continues to test out his warging abilities
 When the group happens upon a weirwood tree, Bran places his hand on its trunk and experiences a series of rapid-fire visions — including glimpses of the three-eyed raven, his father, a snowy Iron Throne in the remains of the Red Keep’s ruined throne room, the fall that paralyzed him, the shadow of a dragon flying over King’s Landing, and a giant weirwood tree — and hears a male voice say, “Look for me, beneath the tree
”When Bran comes out of his trance, he tells the rest of the group that he knows where they have to go


In [104]:
filtered_text = ""
for idx in similar_sentences:
    filtered_text = filtered_text + " " + sentences[idx]
print(filtered_text)

 

Meanwhile, Bran is struggling with the increasingly blurred lines between his own mind and that of his direwolf, Summer, as he continues to test out his warging abilities  When the group happens upon a weirwood tree, Bran places his hand on its trunk and experiences a series of rapid-fire visions — including glimpses of the three-eyed raven, his father, a snowy Iron Throne in the remains of the Red Keep’s ruined throne room, the fall that paralyzed him, the shadow of a dragon flying over King’s Landing, and a giant weirwood tree — and hears a male voice say, “Look for me, beneath the tree ”When Bran comes out of his trance, he tells the rest of the group that he knows where they have to go


In [105]:
get_summary(filtered_text)

[' Bran is struggling with the increasingly blurred lines between his own mind and that of his direwolf, Summer, as he continues to test out his warging abilities. When the group happens upon a weirwood tree, Bran places his hand on its trunk and experiences rapid-fire visions. Bran hears a male voice say, “Look for me, beneath the tree ”']

In [107]:
get_summary(filtered_text, "google/pegasus-xsum")

['In the fifth episode of Game of Thrones’ sixth season, Bran and the Brotherhood Without Banners are sent on a mission to find the three-eyed raven.']